In [1]:
# default_exp field_mapping

# Field Mapping

A big problem with the different neuro measures is that everyone seems to have their own field names for the same measurement. This module is a small collection of utilities for converting between internal and external fields.

In [2]:
# export

import pandas as pd

We often need to load different types of files. Here's something smrt to do that.

In [3]:
# export

def smrt_read(path, fix_id_col = None):
    if path.endswith('xls') or path.endswith('xlsx'):
        return pd.read_excel(path, na_values=['na', '-', 'nd'])
    elif path.endswith('csv'):
        return pd.read_csv(path, na_values=['na', '-', 'nd'])
    elif path.endswith('tsv') or path.endswith('tab'):
        return pd.read_csv(path, sep = '\t', na_values=['na', '-', 'nd'])
    raise ValueError(f'Could not load file {path}')

There's also a need to do a moderate amount of cleaning. Here are some utility functions for common issues.

In [4]:
# export

from functools import reduce
  
# composite_function accepts N
# number of function as an 
# argument and then compose them
def composite_function(*func):
          
    def compose(f, g):
        return lambda x : f(g(x))
              
    return reduce(compose, func, lambda x : x)



def fix_dates(df):
    
    df['date_of_visit'] = pd.to_datetime(df['date_of_visit'])
    return df

def fix_study_ids(df):
    
    ids = []
    for _id in df['study_id']:
        if type(_id) == str:
            _id = int(_id[1:])
        ids.append(_id)
    df['study_id'] = ids
    return df
    

In [25]:
# export

class FieldMapper(object):
    
    def __init__(self, mapping_df, post_convert = None):
        
        self.mapping_df = mapping_df
        if post_convert is None:
            self.post_convert = post_convert
        elif type(post_convert) == list:
            self.post_convert = composite_function(*post_convert)
        else:
            self.post_convert = post_convert
        
    @staticmethod
    def from_file(path, post_convert = None):
        
        if path.endswith('csv'):
            mapping_df = pd.read_csv(path)
        elif path.endswith('tsv'):
            mapping_df = pd.read_csv(path, sep = '\t')
        elif path.endswith('xlsx'):
            mapping_df = pd.read_excel(path)
        else:
            raise ValueError(f'Could not understand {path}')
        
        return FieldMapper(mapping_df, post_convert=post_convert)
    
    def convert(self, data, source_column, target_column, post_convert = True):
        
        assert source_column in self.mapping_df
        assert target_column in self.mapping_df
        
        source_fields = sorted(set(self.mapping_df[source_column].dropna()))
        target_fields = sorted(set(self.mapping_df[target_column].dropna()))
        
        id_df = self.mapping_df[[source_column, target_column]].dropna()
        id_dict = dict(row.values for _, row in id_df.iterrows())
        mapped_data = data.reindex(source_fields, axis=1).rename(columns = id_dict)
        mapped_data = mapped_data.loc[:, ~mapped_data.columns.duplicated()]
        
        #print(target_fields)
        mapped_data = mapped_data.reindex(target_fields, axis=1)
        
        if post_convert:
            if (type(post_convert) == bool) and self.post_convert is not None:
                mapped_data = self.post_convert(mapped_data)
            elif hasattr(post_convert, '__call__'):
                mapped_data = post_convert(mapped_data)
        
        return mapped_data
    
    
    def multi_merge(self, output_id, items, index_keys = ['study_id', 'patient_visit_number']):
        """Merge many files into the same ID system."""
        
        all_data = []
        all_inds = pd.MultiIndex.from_tuples([], names=index_keys)
        for source_id, dset in items:
            if type(dset) == str:
                dset = smrt_read(dset)
            all_data.append(self.convert(dset, source_id, output_id).groupby(index_keys).first())
            
            all_inds = all_inds.union(all_data[-1].index)
        
        final_data = all_data[0].reindex(all_inds, axis=0)
        for dset in all_data[1:]:
            final_data = final_data.combine_first(dset)
            
        return final_data
    
    
    def update_field_mapping(self, *calculators):
        
        new_rows = []
        for calc in calculators:
            for op in calc.operations:
                new_rows.append(op.to_field_mapping())
                
        self.mapping_df = pd.concat([self.mapping_df, pd.DataFrame(new_rows)], 
                                    axis=0, ignore_index=True)
        self.mapping_df = self.mapping_df.groupby('internal_field', as_index = False).last()
                            
        
        
        
        

In [6]:
map_df = pd.DataFrame([{'internal_field': 'fieldA', 'redcap_id': 'field_a', 'cnns_id': 'fielda'},
                       {'internal_field': 'fieldB', 'redcap_id': 'field_b', 'cnns_id': 'fieldb'},
                       {'internal_field': 'fieldC', 'redcap_id': None, 'cnns_id': 'fieldc'},
                       {'internal_field': 'fieldD', 'redcap_id': 'field_d', 'cnns_id': None},])
field_data = pd.DataFrame([{'fieldA': 1, 'fieldB': 2, 'fieldC':3, 'fieldD': 4},
                           {'fieldA': 1, 'fieldB': 2, 'fieldC':3, 'fieldD': 4},
                           {'fieldA': 1, 'fieldB': 2, 'fieldC':None, 'fieldD': None},
                           {'fieldA': 1, 'fieldB': None, 'fieldC':3, 'fieldD': 4}])
field_data

,fieldA,fieldB,fieldC,fieldD
0,1,2.0,3.0,4.0
1,1,2.0,3.0,4.0
2,1,2.0,NaN,NaN
3,1,NaN,3.0,4.0


In [7]:
mapper = FieldMapper(map_df)

out = mapper.convert(field_data, 'internal_field', 'redcap_id')
assert (out.columns == ['field_a', 'field_b', 'field_d']).all()
assert (out['field_a'] == 1).all()
out

,field_a,field_b,field_d
0,1,2.0,4.0
1,1,2.0,4.0
2,1,2.0,NaN
3,1,NaN,4.0


In [8]:
mapper.convert(field_data, 'internal_field', 'cnns_id')

,fielda,fieldb,fieldc
0,1,2.0,3.0
1,1,2.0,3.0
2,1,2.0,NaN
3,1,NaN,3.0


In [9]:
from autoneuro.domains import Collection

In [10]:
cnac = Collection.from_defaults()

In [11]:
calcs = [bat.calculator for dom in cnac.domains.values() for bat in dom.batteries]

In [12]:
cnac.mapper.update_field_mapping(*calcs)

In [17]:
cnac.mapper.mapping_df.to_excel('data/field_mappings_merged.xlsx')

In [14]:
mapper = FieldMapper.from_file('data/field_mappings_KD.xlsx',
                               post_convert = [fix_dates, fix_study_ids])

In [27]:
SHOW_DATA = 'new' # new or old

if SHOW_DATA == 'new':
    paths = [('redcap_id', 'data/ducomout/ClinicalAndTranslati_DATA_2021-04-24_1737.csv'),
             ('cnns_id', 'data/cnns/EXPORT_DD.csv')]
    req_fields = ['cnns_bvmt_t']
else:
    paths = [('oldneuro_id', 'data/neuro_data.xlsx')]
    req_fields = ['bvmt_trial1']


mapper = FieldMapper.from_file('data/field_mappings_merged.xlsx', post_convert=[fix_study_ids, fix_dates])
map_neuro = mapper.multi_merge('internal_field', paths, index_keys = ['study_id', 'date_of_visit']).dropna(subset = req_fields)
map_neuro.head()

/home/will/anaconda3/envs/autoneuro/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (2,17,18,97,106,109,112,139,140,141,247,254,268,275,279,281,357,358,363,410,411,477,494,505,535,536,541,707,709,711,853,870,884,928,929,930,944,949,959,966,977,987,988,989,995,1004,1005,1006,1012,1019,1021,1022,1023,1029) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,,abdominal_girth,age,age_first_drivers_license,age_first_used_drug,agenp,aids_defining_indicat2_2dd,aids_defining_indicators,also_driving_assessment,amitriptyline_end_date,amitriptyline_start_date,...,year_diagnosed_tuberculosi,year_diagnosed_uterine_can,year_diagnosed_with_asthma,year_diagnosed_with_copd,year_diagnosed_with_diabet,year_diagnosed_with_elevat,year_diagnosed_with_hypert,years_seropositive,you_get_angry_at_the_behav,you_intend_to_drive_to_des
study_id,date_of_visit,,,,,,,,,,,,,,,,,,,,,
2,2021-03-11,37.5,70.0,30,22yo,70.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0
5,2020-11-19,51.5,61.0,None,None,61.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,1964.0,2008.0,2010.0,2014.0,NaN,36 years,NaN,NaN
13,2021-02-25,37.0,75.0,18yo,None,75.0,NaN,NaN,1.0,NaN,NaN,...,1996.0,NaN,NaN,2003.0,2019.0,2007.0,2009.0,23 years,0.0,0.0
25,2021-03-16,47.5,63.0,62yo,16yo,63.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,2020.0,2019.0,2009.0,2000.0,21 Years,2.0,1.0
38,2021-01-13,50.0,51.0,30yo,13yo,51.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,1973.0,NaN,NaN,2007.0,2012.0,25 years,0.0,0.0


In [16]:
mapped_data.dropna(subset = ['bvmt_trial1'])

abdominal_girth   age age_first_drivers_license  \
study_id date_of_visit                                                   
1        2014-11-10                NaN  59.0                       NaN   
2        2013-10-22                NaN  62.0                       NaN   
4        2014-11-10                NaN  65.0                       NaN   
5        2013-12-12                NaN  54.0                       NaN   
         2020-11-19               51.5  61.0                      None   
...                                ...   ...                       ...   
733      2020-01-29               47.0  64.0                      None   
742      2020-02-27               45.0  41.0                      None   
757      2021-01-12               41.0  48.0                      35yo   
759      2021-01-27               36.5  43.0                      None   
761      2021-02-09               43.5  44.0                        18   

                       age_first_used_drug  agenp  aids_defining_indicat2_2dd  \
study_id date_of_visit                                                          
1        2014-11-10                    NaN    NaN                         NaN   
2        2013-10-22                    NaN    NaN                         NaN   
4        2014-11-10                    NaN    NaN                         NaN   
5        2013-12-12                    NaN    NaN                         NaN   
         2020-11-19                   None   61.0                         NaN   
...                                    ...    ...                         ...   
733      2020-01-29                     23    NaN                         NaN   
742      2020-02-27                   None    NaN                         NaN   
757      2021-01-12                   None   48.0                         NaN   
759      2021-01-27                   None   43.0                         NaN   
761      2021-02-09                   16yo   45.0                         NaN   

                        aids_defining_indicators  also_driving_assessment  \
study_id date_of_visit                                                      
1        2014-11-10                          NaN                      NaN   
2        2013-10-22                          NaN                      NaN   
4        2014-11-10                          NaN                      NaN   
5        2013-12-12                          NaN                      NaN   
         2020-11-19                          NaN                      0.0   
...                                          ...                      ...   
733      2020-01-29                          NaN                      NaN   
742      2020-02-27                          NaN                      NaN   
757      2021-01-12                          NaN                      1.0   
759      2021-01-27                          NaN                      1.0   
761      2021-02-09                          NaN                      1.0   

                        amitriptyline_end_date  amitriptyline_start_date  ...  \
study_id date_of_visit                                                    ...   
1        2014-11-10                        NaN                       NaN  ...   
2        2013-10-22                        NaN                       NaN  ...   
4        2014-11-10                        NaN                       NaN  ...   
5        2013-12-12                        NaN                       NaN  ...   
         2020-11-19                        NaN                       NaN  ...   
...                                        ...                       ...  ...   
733      2020-01-29                        NaN                       NaN  ...   
742      2020-02-27                        NaN                       NaN  ...   
757      2021-01-12                        NaN                       NaN  ...   
759      2021-01-27                        NaN                       NaN  ...   
761      2021-02-09             